In [5]:
!gdown --id 152sWECukjvLerrVG2NUO8gtMFg83RKCF  --output data/WISDM_ar_latest.tar.gz

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=152sWECukjvLerrVG2NUO8gtMFg83RKCF
To: /home/udaygirish/Projects/learn_interviews/time_series_experiments/data/WISDM_ar_latest.tar.gz
100%|██████████████████████████████████████| 11.4M/11.4M [00:00<00:00, 23.2MB/s]


In [6]:
! tar -xvf data/WISDM_ar_latest.tar.gz -C data

WISDM_ar_v1.1/
WISDM_ar_v1.1/readme.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw_about.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_transformed.arff
WISDM_ar_v1.1/WISDM_ar_v1.1_trans_about.txt


In [9]:
# Import libraries 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import sys 
import seaborn as sns

In [8]:
SEED = 45

In [10]:
# Import DL libraries
import tensorflow as tf
from tensorflow import keras


In [11]:
# Import Traditional ML Libraries 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
# ToDO:
# 1. Load the data
# 2. Preprocess the data - Build a Pipeline (Common Data Pipeline)
# 3. Processed Data - Pipeline - Traditional ML Model
# 4. Processed Data - Pipeline - Deep Learning Model
# 5. Post Processing of Results
# 6. Model Evaluation 
# 7. API to send a pack and get the results analysed
# 8. Deploy on Cloud - Package the model and API -Docker
# 9. Optimize the model for Edge - (Tf Lite, ONNX, .h )
# 10. Sample Input and Output Data representations